In [48]:
# Import the required libraries and dependencies

import os
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi
from MCForecastTools import MCSimulation
%matplotlib inline

# import SDK libraries from OpenSea / MagicEden if/when available



In [49]:
# Load the environment variables from the .env file
#by calling the load_dotenv function
load_dotenv()

True

In [ ]:
# Set the Free Crypto API Call endpoint URLs for the held cryptocurrency assets

btc_url = "https://api.alternative.me/v2/ticker/Bitcoin/?convert=USD"
eth_url = "https://api.alternative.me/v2/ticker/Ethereum/?convert=USD"


In [ ]:
# Make an API call to access the current price of BTC

btc_response = requests.get(btc_url).json()

# Display the response data
print(json.dumps(btc_response, indent=4, sort_keys=True))

In [ ]:
# Make an API call to access the current price ETH
eth_response = requests.get(eth_url).json()

# Display the response data
print(json.dumps(eth_response, indent=4, sort_keys=True))

In [ ]:
# Navigate the BTC response object to access the current price of BTC
btc_price = btc_response["data"]["1"]["quotes"]["USD"]["price"]

# Print the current price of BTC
btc_price

In [ ]:
# Navigate the BTC response object to access the current price of ETH
eth_price = eth_response["data"]["1027"]["quotes"]["USD"]["price"]

# Print the current price of ETH
eth_price

In [ ]:
# Set the current amount of shares held in both the stock (SPY) and bond (AGG) portion of the portfolio.
spy_shares = 110
agg_shares = 200

In [ ]:
# Set the variables for the Alpaca API and secret keys
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca tradeapi.REST object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [ ]:
# Set the tickers for both the bond and stock portion of the portfolio
tickers = ["SPY", "AGG"]

# Set timeframe to 1Day
timeframe = "1Day"

# Format current date as ISO format
# Set both the start and end date at the date of your prior weekday 
# This will give you the closing price of the previous trading day
# Alternatively you can use a start and end date of 2020-08-07
start_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2020-08-07", tz="America/New_York").isoformat()

In [ ]:
# Use the Alpaca get_bars function to get current closing prices the portfolio
# Be sure to set the `df` property after the function to format the response object as a DataFrame
prices_df = alpaca.get_bars(
    tickers,
    timeframe,
    start=start_date,
    end=end_date
).df

# Reorganize the DataFrame
# Separate ticker data
SPY = prices_df[prices_df['symbol']=='SPY'].drop('symbol', axis=1)
AGG = prices_df[prices_df['symbol']=='AGG'].drop('symbol', axis=1)

# Concatenate the ticker DataFrames
prices_df = pd.concat([SPY, AGG], axis=1, keys=["SPY", "AGG"])

# Review the first 5 rows of the Alpaca DataFrame
prices_df.head()

In [ ]:
# Access the closing price for AGG from the Alpaca DataFrame
# Converting the value to a floating point number
agg_close_price = float(prices_df["AGG"]["close"])

# Print the AGG closing price
agg_close_price

In [ ]:
# Access the closing price for SPY from the Alpaca DataFrame
# Converting the value to a floating point number
spy_close_price = float(prices_df["SPY"]["close"])

# Print the AGG closing price
spy_close_price

In [ ]:
# Request daily_returns data for a collection from OpenSea API

# os_response = requests.get(<INSERT OS ENDPOINT>).json()



In [ ]:
# Request daily_returns data for a collection from MagicEden API

# me_response = requests.get(<INSERT ME ENDPOINT>).json()



In [ ]:
# Calculate the pct_change of OS and ME dataframes


In [ ]:
# Calculate the covariance, beta, and sharpe ratios of OS and ME dataframes


In [ ]:
# Perform Monte Carlo simulations for OS and ME data

In [89]:
# Set the base url for the NFTPort API
base_nftport_url = "https://api.nftport.xyz/v0"

# Set the headers, including for authentication
token = os.getenv("NFTPORT_API_KEY")
headers = {'Authorization': token}


In [159]:
def get_nfts_for_account(chain, account):
  
    response = requests.get(base_nftport_url + "/accounts/" + account + f"?chain={chain}", headers=headers).json()
    return response
    
def get_transactions_for_nft(chain, trans_type, contract_address, token_id):
    
    response = requests.get(base_nftport_url + "/transactions/nfts/" + contract_address + "/" + token_id + f"?chain={chain}&type={trans_type}", headers=headers).json()
    return response
    

In [195]:

# Call function to get transactions, returns json
response = get_transactions_for_nft("ethereum", "sale", "0x0eff88e66eb717f8f7e0701345f1cc119b487f86", "4849")

# create a dataframe from the response data
response_df = pd.DataFrame(response)
display(response_df)

# Flatten the json to read the transactions into a dataframe
transactions_df = pd.json_normalize(response, record_path =['transactions'])

transactions_df['transaction_date'] = pd.to_datetime(transactions_df['transaction_date'])
transactions_df.set_index('transaction_date')

# display(transactions_df.info())
# display(transactions_df.sort_values(by=['transaction_date']))

transactions_df["sale_returns"] = transactions_df["price_details.price_usd"].pct_change(periods = 1)
display(transactions_df)


,response,transactions
0,OK,"{'type': 'sale', 'buyer_address': '0xea70f5fcf..."
1,OK,"{'type': 'sale', 'buyer_address': '0xf4b746a0a..."
2,OK,"{'type': 'sale', 'buyer_address': '0x77d7a0237..."


,type,buyer_address,seller_address,quantity,transaction_hash,block_hash,block_number,transaction_date,marketplace,nft.contract_type,nft.contract_address,nft.token_id,price_details.asset_type,price_details.price,price_details.price_usd,sale_returns
0,sale,0xea70f5fcf0e51900d13b7992b0996530f036a133,0xf4b746a0ae12ff4f9854aaa712518193bf9e0ffa,1,0x7aa120761e26abb56f27182638e71ec95d5e46de3ae9...,0x96ad219b64abf6fb8d2ab022d9af212b5d441a1d16ce...,14198978,2022-02-13 16:58:44,opensea,ERC721,0x0eff88e66eb717f8f7e0701345f1cc119b487f86,4849,ETH,0.128,400.650956,NaN
1,sale,0xf4b746a0ae12ff4f9854aaa712518193bf9e0ffa,0x77d7a0237819072e0ead5e754aaa4edef35bcfcf,1,0xdba90a88d91879ecff9e13c33f90116ec9396eabf587...,0x63c5710633b52779c22bdeb4b48c71916a8d69c4d4f3...,14194745,2022-02-13 01:17:05,opensea,ERC721,0x0eff88e66eb717f8f7e0701345f1cc119b487f86,4849,ETH,0.140,431.379256,0.076696
2,sale,0x77d7a0237819072e0ead5e754aaa4edef35bcfcf,0x700541b878657623787c87cc32a1ba7be49981d1,1,0xbb1eb527945c214d2bdb4c9458c801e18a2a86242c04...,0x948bc981d8a3a16d37ab9a2cd60647c2707f1fb68219...,14191117,2022-02-12 12:03:26,opensea,ERC721,0x0eff88e66eb717f8f7e0701345f1cc119b487f86,4849,ETH,0.100,301.333140,-0.301466
